##**Trabajo de Fin de Máster**
Salomé A. Sepúlveda Fontaine

##**Preparación del entorno**

####Importación de Librerías

In [1]:
# Data Analysis
import pandas as pd
import numpy as np
from numpy import asarray
from numpy import savetxt
from numpy import loadtxt
import pickle as pkl
from scipy import sparse

# Data Visualization
import seaborn as sns
import matplotlib.pyplot as plt
import wordcloud
from wordcloud import WordCloud, STOPWORDS

# Text Processing
import re
import itertools
import string
import collections
from collections import Counter
from sklearn.preprocessing import LabelEncoder
import nltk
from nltk.classify import NaiveBayesClassifier
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer

# Machine Learning packages
import sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
import sklearn.cluster as cluster
from sklearn.manifold import TSNE

# Model training and evaluation
from sklearn.model_selection import train_test_split

#Models
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.utils.class_weight import compute_sample_weight
from xgboost import XGBClassifier
from xgboost import plot_importance
from sklearn.base import BaseEstimator, ClassifierMixin

#Metrics
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error, accuracy_score, balanced_accuracy_score
from sklearn.metrics import precision_score, recall_score, f1_score, multilabel_confusion_matrix, confusion_matrix
from sklearn.metrics import classification_report

import nltk
from nltk.corpus import stopwords


# Ignore noise warning
import warnings
warnings.filterwarnings("ignore")

####Funciones predefinidas, explícitas y implícitas.
Análisis, visualizacion y procesamiento.

In [2]:
# Se crea diccionario con las iniciales de cada tipo de personalidad
mbti = {'I':'Introversion', 'E':'Extroversion', 'N':'Intuition', 
        'S':'Sensing', 'T':'Thinking', 'F': 'Feeling', 
        'J':'Judging', 'P': 'Perceiving'}
def var_row(row):
    l = []
    for i in row.split('|||'):
        l.append(len(i.split()))
    return np.var(l)
    
def unique_words(a, b, numWords):
  ## Palabras únicas tipo 'a'
  words_a = (XX[X[a] == 1].mean() / XX[X[b] == 1].mean()).sort_values().rename(lambda x: x[2:]).tail(numWords)
  print('* Unique ' + mbti[a] + ' Words:')
  print(words_a)
  ## Palabras únicas tipo 'b'
  words_b = (XX[X[b] == 1].mean() / XX[X[a] == 1].mean()).sort_values().rename(lambda x: x[2:]).tail(numWords)
  print('\n* Unique ' + mbti[b] + ' Words:')
  print(words_b)

  ## Representación en gráfico de barras
  plt.figure(figsize=(10, 6))
  plt.subplot(131)
  words_a.plot.barh()
  plt.title(mbti[a] + ' vs ' + mbti[b])
  plt.subplot(133)
  words_b.plot.barh()
  plt.title(mbti[b] + ' vs ' + mbti[a])
  plt.show()

def new_Words(a, b, numWords):
  ## Palabras únicas tipo 'a'
  words_a = (testX[X[a] == 1].mean() / testX[X[b] == 1].mean()).sort_values().rename(lambda x: x[2:]).tail(numWords)
  print('* Unique ' + mbti[a] + ' Words:')
  print(words_a)
  ## Palabras únicas tipo 'b'
  words_b = (testX[X[b] == 1].mean() / testX[X[a] == 1].mean()).sort_values().rename(lambda x: x[2:]).tail(numWords)
  print('\n* Unique ' + mbti[b] + ' Words:')
  print(words_b)

  ## Representación en gráfico de barras
  plt.figure(figsize=(10, 6))
  plt.subplot(131)
  words_a.plot.barh()
  plt.title(mbti[a] + ' vs ' + mbti[b])
  plt.subplot(133)
  words_b.plot.barh()
  plt.title(mbti[b] + ' vs ' + mbti[a])
  plt.show()

def preprocess_text(df, remove_special=True):
    texts = df['posts'].copy()
    labels = df['type'].copy()

    #Remove links 
    df["posts"] = df["posts"].apply(lambda x: re.sub(r'https?:\/\/.*?[\s+]', '', x.replace("|"," ") + " "))
    
    #Keep the End Of Sentence characters
    df["posts"] = df["posts"].apply(lambda x: re.sub(r'\.', ' EOSTokenDot ', x + " "))
    df["posts"] = df["posts"].apply(lambda x: re.sub(r'\?', ' EOSTokenQuest ', x + " "))
    df["posts"] = df["posts"].apply(lambda x: re.sub(r'!', ' EOSTokenExs ', x + " "))
    
    #Strip Punctation
    df["posts"] = df["posts"].apply(lambda x: re.sub(r'[\.+]', ".",x))

    #Remove multiple fullstops
    df["posts"] = df["posts"].apply(lambda x: re.sub(r'[^\w\s]','',x))

    #Remove Non-words
    df["posts"] = df["posts"].apply(lambda x: re.sub(r'[^a-zA-Z\s]','',x))

    #Convert posts to lowercase
    df["posts"] = df["posts"].apply(lambda x: x.lower())

    #Remove multiple letter repeating words
    df["posts"] = df["posts"].apply(lambda x: re.sub(r'([a-z])\1{2,}[\s|\w]*','',x)) 

    #Remove very short or long words
    df["posts"] = df["posts"].apply(lambda x: re.sub(r'(\b\w{0,3})?\b','',x)) 
    df["posts"] = df["posts"].apply(lambda x: re.sub(r'(\b\w{30,1000})?\b','',x))

    #Remove MBTI Personality Words - crutial in order to get valid model accuracy estimation for unseen data. 
    if remove_special:
        pers_types = ['INFP' ,'INFJ', 'INTP', 'INTJ', 'ENTP', 'ENFP', 'ISTP' ,'ISFP' ,'ENTJ', 'ISTJ','ENFJ', 'ISFJ' ,'ESTP', 'ESFP' ,'ESFJ' ,'ESTJ']
        pers_types = [p.lower() for p in pers_types]
        p = re.compile("(" + "|".join(pers_types) + ")")
    
    return df

Loading Dataset

To upload the dataset you can use Google Drive with the code below or download the file [mbti_1.csv.zip](https://github.com/ApusDT/MBTI-Personality-Predictor-using-Machine-Learning/blob/main/mbti_1.csv.zip) and unzip it.


In [5]:
from google.colab import drive
drive.mount('/content/drive')
df=pd.read_csv("path/to/file", encoding='utf-8')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


##**Analisis, descripción y visualización de datos.**

Visualización del DataFrame.
Observamos las 4 primeras filas del conjunto de datos.

In [ ]:
df.head()


A modo de ejmplo se visualiz la información de una columna. 

In [ ]:
df.loc[1,:]

Visualización de post.

Como los posts son muchos y no se aprecian en las celdas anteriores, se imprimen en pantalla todos los posts de una fila. Notar que cada post está seprado por "**|||**"

In [ ]:
df.loc[1,'posts']

Descripción de cantidad y tipo de columnas.

In [ ]:
print(df.columns)
print("Dataframe con {} filas y {} columnas".format(df.shape[0],df.shape[1]))
print("Las columnas son: {} y {}".format(df.columns[0],df.columns[1]))

Búsqueda de valores nulos.

In [ ]:
print(df.info())
#contenio de nulls
print("\nNumero de valores nulos")
print(df.isna().sum())

Con el método `describe`  se obtiene una visión global de frecuencias y tipos.
  

*   Hay 16 tipos de indicadores de personalidad únicas, con 1832 ocurrencias.
*   No se observan posts repetidos.

 
 

In [ ]:
df.describe(include=['object'])

Se buscan los valores únicos de las columnas del tipo de personalidad.

In [ ]:
types = np.unique(np.array(df['type']))
types

Se usará el método `groupby` para separar los distintos grupos de personalidad y hacer un análisis más exhaustivo.

In [ ]:
total = df.groupby(['type']).count().sort_values(by=['posts'],ascending=False)
total

**Visualización del número de posts por cada tipo de personalidad.**

Se observa que los datos están desbalanceados en el sentido que no existe una distribución equitativa para el número de post por personalidad, como puede apreciarse en el siguiente gráfico.

In [ ]:
plt.figure(figsize = (12,4))
plot = plt.bar(np.array(total.index), height = total['posts'])
plot[0].set_color('y')
plot[-1].set_color('r')
plt.xlabel('Personality types', size = 14)
plt.ylabel('No. of posts available', size = 14)
plt.title('Total de posts por cada tipo de personalidad')

**SWARM PLOT** :

Swarm Plots, también llamados beeswarm plots, grafican todos los puntos de la data para grupo de personalidad.

In [ ]:
df['words_per_comment'] = df['posts'].apply(lambda x: len(x.split())/50)
df['variance_of_word_counts'] = df['posts'].apply(lambda x: var_row(x))

plt.figure(figsize=(15,10))
sns.swarmplot("type", "words_per_comment", data=df)

**Longitud de publicaciones.**

**DISTANCE PLOT:**

Con seaborn, usando este método, se observa el histograma de la distribución de los datos, por cada columna.
*   Se observa que la mayoría de los posts tienen una longitud de entre 7000 y 9000 palabras.
*   La línea observada representa el núcleo de la densidad de estimación. 
*   Es un problema fundamental de suavizado de datos donde se realizan inferencias sobre la población, basadas en una muestra de datos finitos. Esta estimación de la densidad del núcleo es una función definida como la suma de una función del núcleo en cada punto de datos.








In [ ]:
df["length_posts"] = df["posts"].apply(len)
sns.distplot(df["length_posts"]).set_title("Distribución del largo de todos los 50 posts")

**Usuarios y posts**

Con la función `extract` se cuenta el número de usuarios y posts presentes en el dataset. Se concluye:


*   Hay muchos hipervínculos presentes en la data.
*   Se asume que los hipervínculos no proveen ninguna información real acerca de la personalidad de los usuarios.
*   Se prescinde de ellos para los análisis posteriores.
Ha de tenerse en cuenta que los ejemplos dados vienen de los usuarios que han dejado comentarios; así, los modelos serán aplicados solamente a ellos (los que han dejado comentarios).




Con este *sesgo* nuestros modelos podrían fallar en clasificar las personalidades. Se comprobará más adelante.

In [ ]:
def extract(posts, new_posts):
    for post in posts[1].split("|||"):
        new_posts.append((posts[0], post))

posts = []
df.apply(lambda x: extract(x, posts), axis=1)
print("Number of users", len(df))
print("Number of posts", len(posts))
print("5 posts from start are:")
posts[0:5]

In [ ]:
words = list(df["posts"].apply(lambda x: x.split()))
words = [x for y in words for x in y]
Counter(words).most_common(10)

**WORDCLOUD**
**debería poner esto en parte de PLN??????**
WordCloud es una técnica que permite mostrar las palabras que son más frecuentes en un texto dado.


Se observa una perspectiva general del las palabras más frecuentes, en todo el dataset.

In [ ]:
#Plotting the most common words with WordCloud.
wc = wordcloud.WordCloud(width=1200, height=500, 
                         collocations=False, background_color="white", 
                         colormap="tab20b").generate(" ".join(words))

# collocations to False  is set to ensure that the word cloud doesn't appear as if it contains any duplicate words
plt.figure(figsize=(25,10))
# generate word cloud, interpolation 
plt.imshow(wc, interpolation='bilinear')
_ = plt.axis("off")

Se muestran las palabras más comunes por cada tipo de personalidad.

In [ ]:
fig, ax = plt.subplots(len(df['type'].unique()), sharex=True, figsize=(15,len(df['type'].unique())))
k = 0
for i in df['type'].unique():
    df_4 = df[df['type'] == i]
    wordcloud = WordCloud(max_words=1628,relative_scaling=1,normalize_plurals=False).generate(df_4['posts'].to_string())
    plt.subplot(4,4,k+1)
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.title(i)
    ax[k].axis("off")
    k+=1

Anteriormente se ha visto que la carga de los datos no es balanceada (no se tiene igual número de muestras para cada personalidad). Se muestran a continuación, para cada post, su representación por dimensiones. Se sabe que las dimensiones son excluyentes, así por ejemplo un post puede pertenecer o bien a N o bien a S. Esta pertenencia se construye a través de una representación binaria, donde 0 será la no pertenencia y 1 la pertenencia.

In [ ]:
def get_types(row):
    t = row['type']

    # Dimensiones: IE, NS, TF, JP
    ## IE
    I = 1 if t[0] == 'I' else 0 
    ## NS
    N = 1 if t[1] == 'N' else 0 
    ## TF
    T = 1 if t[2] == 'T' else 0 
    ## JP
    J = 1 if t[3] == 'J' else 0 

    # Ejemplo: INFP (I = 1, N = 1, T = 0, J = 0)

    return pd.Series( {'IE':I, 'NS':N , 'TF': T, 'JP': J }) 

data_new = df.join(df.apply (lambda row: get_types (row),axis=1))
data_new.head(5)

A continuación se genera una cuenta del total de post para cada dimensión.

In [ ]:
print ("Introversion (I) -  Extroversion (E):\t", data_new['IE'].value_counts()[1], " / ", data_new['IE'].value_counts()[0])
print ("Intuition (N) – Sensing (S):\t\t", data_new['NS'].value_counts()[1], " / ", data_new['NS'].value_counts()[0])
print ("Thinking (T) – Feeling (F):\t\t", data_new['TF'].value_counts()[1], " / ", data_new['TF'].value_counts()[0])
print ("Judging (J) – Perceiving (P):\t\t", data_new['JP'].value_counts()[1], " / ", data_new['JP'].value_counts()[0])

A continuación se obtiene la representación gráfica de el conteo anterior.

In [ ]:
N = 4

bott = (data_new['IE'].value_counts()[0], data_new['NS'].value_counts()[0], data_new['TF'].value_counts()[0], data_new['JP'].value_counts()[0])
top = (data_new['IE'].value_counts()[1], data_new['NS'].value_counts()[1], data_new['TF'].value_counts()[1], data_new['JP'].value_counts()[1])

ind = np.arange(N)    # the x locations for the groups
width = 0.4      # the width of the bars: can also be len(x) sequence

plt.figure(figsize=(4,3))
p1 = plt.bar(ind, bott, width, color='cornflowerblue')
p2 = plt.bar(ind, top, width, bottom=bott, color='aquamarine')

plt.ylabel('Número de muestras')
plt.title('Personalidades agrupadas en dimensiones')
plt.xticks(ind, ('I/E',  'N/S', 'T/F', 'J/P',))

plt.show()

##**Preprocesamiento de datos.**

**Estandarización de Datos**
Vínculos y signos.
Se procede a hacer un conteo de vínculos, signos de exclamación y signos de interrogación por cada tipo de personalidad.


*   Se sustituye cada hipervínculo por la palabra LINK
*   Se comprueba la existencia de signos de exclamación contiguos a direcciones web.



In [ ]:
# Creación de una nueva columna para contener la información modificada de 'posts'
df['posts_noLink'] = df['posts']

import re
for i in range(0, 8675):
  df['posts_noLink'][i] = re.sub(r'http\S+', 'LINK', df['posts'][i]);

# Creación de las columnas 'links', 'qns' y 'exclams' para contener el total de enlaces, preguntas y exclamaciones, respectivamente
df['links'] = df['posts'].apply(lambda x: x.count('http')); 
df['qns'] = df['posts_noLink'].apply(lambda x: x.count('?')); 
df['exclams'] = df['posts_noLink'].apply(lambda x: x.count('!')); 
df['words'] = df['posts'].apply(lambda x: len(x.split()))
df['comments'] = df['words']
for i in range (0, 8675):
  df['comments'][i] = len(df['posts'][i].split('|||'))
print(df.columns)
df.head(10)


**Medias aritméticas de vínculos, signos y palabras.**

Con el método `groupBy` se grafica a continuación, la media aritmética de hipervínculos, signos de exclamación y signos de interrogación, por cada tipo de personalidad.

In [ ]:
groupByColumns = df.groupby('type').agg({'links':'mean','qns':'mean','exclams':'mean'})

plot = groupByColumns.plot(kind = 'bar', title = 'Media artimética de hipervínculos, interrogaciones y exclamaciones por tipo de personalidad')
plot.set_xlabel("")
plot.set_ylabel("")

A continuación, se observa la gráfica de la media artimética de palabras, por tipo de personalidad.

In [ ]:
groupByWords = df.groupby('type').agg({'words':'mean'})
print("Media de palabras {}".format(np.mean(groupByWords)))
plot2 = groupByWords.plot(kind = 'bar', title = 'Media de palabras por tipo de personalidad')
plot2.set_xlabel("")
plot2.set_ylabel("")

##**Procesamiento de lenguaje natural**
Haciendo uso de la librería nltk, Natural Language Toolkit o en español, Kit de herramientas de lenguaje natural; se estudian las *stopwords* es decir, palabras de alta frecuencia como the, to, dentro del documento antes de procesarlo.
Las palabras irrelevantes suelen tener poco contenido léxico y su presencia en un texto no lo distingue de otros textos.

In [ ]:
nltk.download('stopwords')

stop_words = set(stopwords.words('english')); stop_words

Con las *stopwords* encontradas anteriormente y usando `CountVectorizer`  se encuentran las frecuencias de las palabras de la columna posts.

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(analyzer="word", max_features=2000, strip_accents='ascii', stop_words=stop_words)
# Frecuencia de cada una de las palabras de la columna 'posts'
X_cnt = cv.fit_transform(df['posts'])
# Diccionario de palabras
cv.vocabulary_.keys()

**Categorización de las personalidades.**
Se procede a diferenciar cada usuario de acuerdo a uno de los cuatro elementos que forma cada tipo de personalidad; en su forma binaria (1 pertence, 0 no pertenece). 

In [ ]:
X = pd.DataFrame()
for c in 'IENSTFJP':
    X[c] = df['type'].apply(lambda x: 1 if c in x else 0)

X

Se transforma lo anterior a un array.


In [ ]:
X_cnt.toarray()

Se presenta un DataFrame en código binario para cada usuario, detallando la pertenencia o presencia de los elmentos identificados en hipervínculos.

In [ ]:
pd.DataFrame(X_cnt.toarray(), columns=['w_' + k for k in cv.vocabulary_.keys()])

Al DataFrame **X** creado anteriormente, se le añaden las columnas anteriores.

In [ ]:
X = pd.concat([X, pd.DataFrame(X_cnt.toarray(), columns=['w_' + k for k in cv.vocabulary_.keys()])],
              axis=1)
X

Se verifican las columnas que empiezan con la letra w que hace referencia a la palabra que se está tratando.

In [ ]:
wcols = [col for col in X.columns if col.startswith('w_')]; wcols
#X[wcols].mean() >= 0.5


Frecuencia de cada palabra



In [ ]:
# Selección de las palabras con frecuencia superior a 50% y trasposición entre filas y columnas
# La trasposición es requerida porque la condición sobre las frecuencias devuelve True/False para cada palabra
# en una misma  fila. Si no se realiza, no se puede aplicar dicha condición.
X[wcols].T[X[wcols].mean() >= 0.5]

# Trasposición de nuevo para volver a tener las palabras como columnas 
X[wcols].T[X[wcols].mean() >= 0.5].T

# Creación de un nuevo DataFrame a partir del anterior que contenga las palabras más usadas.
XX = X[wcols].T[X[wcols].mean() >= 0.5].T; XX

Se calcula, a modo de ejemplo, la frecuencia media de las palabras citadas para el tipo E (extrovertido) y luego para el tipo I (Introvertido)

In [ ]:
print('Frequencia de palabras para EXTROVERSION:')
words_E = XX[X['E'] == 1].mean(); words_E

In [ ]:
print('Frequencia de palabras para INTROVERSION:')
words_I = XX[X['I'] == 1].mean(); words_I

Comparativa de palabras en Extroversión versus palabras en Introversión. Se diferencian 3 tipos de cocientes:


*   Cociente que tiende a 0: significa que hay mucho mayor cantidad de palabras para el tipo Introversión.
*   Cociente menor que 1: el número de palabras se asemeja para ambos tipos de personalidad.
*   Cociente mayor que 1: el número  de palabras es mayor para el tipo Extroversión.






In [ ]:
print('EXTROVERSION vs INTROVERSION Words')
(words_E/words_I).rename(lambda x: x[2:])

Con la función `unique_words` se obtienen las palabras únicas más usadas por cada eje dicotómico de personalidad, a saber: 


*   I-E: Inroversión - Extroversión
*  N-S: Intuition - Sensation
*   T-F: Thinking-Feeling
*   J-P: Judging-Perceiving

Una vez contabilizadas, se grafican las comparaciones.








In [ ]:
# Extroversion and Introversion
unique_words('E', 'I', 15)

In [ ]:
# Intuition and Sensing
unique_words('N', 'S', 15)

In [ ]:
# Thinking and Feeling
unique_words('T', 'F', 15)

In [ ]:
# Judging and Perceiving
unique_words('J', 'P', 15)

**Análisis del coeficiente de correlación de Pearson con respecto al Procesamiento de Lenguaje Natural.**

Después de la categorización de los datos, después del análisis de personalidades; para determinar de forma visual el análisis de relación entre las características de personlidades, se construye un mapa de calor para representar la colinealidad entre las característcas mencionadas.

A través del coeficiente de Pearson, se verá la correlación entre los distintos descriptores que dan lugar a los 16 tipos de personalidad.

In [ ]:
# Desglose de cada personalidad en las características que contiene
dfX = pd.DataFrame()
for c in 'IENSTFJP':
    dfX[c] = df['type'].apply(lambda x: 1 if c in x else 0)
corr = dfX[['I','E','N','S','T','F','J','P']].corr(); corr
cmap = plt.cm.RdBu
plt.figure(figsize=(12,10))
plt.title('Matriz de correlación de Pearson', size=15)
sns.heatmap(corr, cmap=cmap,  annot=True, linewidths=1)

Se continúa con el procesamiento de lenguaje natural.

Se emplea el método TF_IDF para calcular la frecuencia de cada una de las palabras a partir del objeto TfidfTransformer, el cual generará un vector que será empleado para entrenar un modelo de procesamiento. 

Se preparan los datos que serán utilizados en los modelos de ML.

In [ ]:
# Recuperación de la información del objeto CountVectorizer
feature_names = list(enumerate(cv.get_feature_names()))
X_cnt.toarray()

# Inicialización del objeto tf-idf para el cálculo de las frecuencias
tfizer = TfidfTransformer()

# Obtención de las frecuencias a partir del objeto CountVectorizer
X_tfidf = tfizer.fit_transform(X_cnt).toarray()

Y = X.filter(['I','E','N','S','T','F','J','P'], axis=1)

# Añadido de las palabras y frecuencias al DataFrame con el desglose de personalidades
testX = pd.concat([Y, pd.DataFrame(X_tfidf, columns=['w_' + k for k in cv.vocabulary_.keys()])],
              axis=1)


Se observan las palabras específicas con relación a cada uno de los ejes dicotómicos.

In [ ]:
# Extroversion vs Introversion
new_Words('E', 'I', 15)

In [ ]:
# Thinking vs Feeling
new_Words('T', 'F', 15)

In [ ]:
# Judging vs Perceiving
new_Words('J', 'P', 15)

Se procede a un preprocesamiento donde se eliminan las publicaciones con menos de X palabras y posteriormente se crea un DataFrame con la información obtenida.

In [ ]:
#Preprocesamiento del texto introducido
new_df = preprocess_text(df)

#Eliminación de publicaciones con menos de X palabras
min_words = 15
print("Before : Numbero de publicaciones", len(new_df)) 
new_df["no. of. words"] = new_df["posts"].apply(lambda x: len(re.findall(r'\w+', x)))
new_df = new_df[new_df["no. of. words"] >= min_words]

print("Después : Numbero de publicaciones", len(new_df))

In [ ]:
new_df.head()

**LabelEncoder**: proporcionado por la biblioteca Sklearn que convierte los niveles de características categóricas (etiquetas) en forma numérica para convertirla en la forma legible por máquina. Codifica etiquetas con un valor entre 0 y n_classes-1 donde n es el número de etiquetas distintas. Si una etiqueta se repite, asigna el mismo valor al que se asignó anteriormente.

Se muestra la conversión de la personalidad de MBTI (o objetivo o función Y) en forma numérica mediante la codificación de etiquetas con`LabelEncoder`  y se guarda en el DataFrame antes creado.


In [ ]:
# Codificación del tipo de personalidad
enc = LabelEncoder()
new_df['type of encoding'] = enc.fit_transform(new_df['type'])

target = new_df['type of encoding']

In [ ]:
new_df.head()

**CountVectorizer**  se usa para convertir una colección de documentos de texto en un vector de conteos de términos / tokens y construir un vocabulario de palabras conocidas, pero también para codificar nuevos documentos usando ese vocabulario. También permite el preprocesamiento de datos de texto antes de generar la representación vectorial.

Se usa `stop_words = 'english'`  con CountVectorizer ya que esto sólo cuenta las ocurrencias de cada palabra en su vocabulario, palabras extremadamente comunes como 'the', 'and', etc. se convertirán en características muy importantes mientras agregan poco significado para el texto. Este es un paso importante en el procesamiento previo, ya que nuestro modelo a menudo se puede mejorar si no tiene en cuenta esas palabras.

-  Se vectorizan las publicaciones para el modelo y se filtran palabras vacías.
-  Conversión de publicaciones (o características X) en forma numérica usando vectorización de conteo

In [ ]:
# Vectorización por stop_words
vect = CountVectorizer(stop_words='english') 

# Conversión de publicaciones
train =  vect.fit_transform(new_df["posts"])

####**hasta aquí PLN**

Se divide el conjunto de datos en entrenamiento y validación.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(train, target, test_size=0.4, stratify=target, random_state=42)
print ((X_train.shape),(y_train.shape),(X_test.shape),(y_test.shape))

In [ ]:
accuracies = {}

#**Implementación de algoritmos en Hard Computing**



####Definición de funciones 
A continuación definen una serie de funciones que será usadas para la implementación de los modelos.
Se generan clases genéricas para visualización de resultados.



In [ ]:
class FuzzyKNN(BaseEstimator, ClassifierMixin):
	def __init__(self, k=3, plot=False):
		self.k = k
		self.plot = plot


	def fit(self, X, y=None):
		self._check_params(X,y)
		self.X = X
		self.y = y

		self.xdim = len(self.X[0])
		self.n = len(y)

		classes = list(set(y))
		classes.sort()
		self.classes = classes

		self.df = pd.DataFrame(self.X)
		self.df['y'] = self.y

		self.memberships = self._compute_memberships()

		self.df['membership'] = self.memberships

		self.fitted_ = True
		return self


	def predict(self, X):
		if self.fitted_ == None:
			raise Exception('predict() called before fit()')
		else:
			m = 2
			y_pred = []

			for x in X:
				neighbors = self._find_k_nearest_neighbors(pd.DataFrame.copy(self.df), x)

				votes = {}
				for c in self.classes:
					den = 0
					for n in range(self.k):
						dist = np.linalg.norm(x - neighbors.iloc[n,0:self.xdim])
						den += 1 / (dist ** (2 / (m-1)))

					neighbors_votes = []
					for n in range(self.k):
						dist = np.linalg.norm(x - neighbors.iloc[n,0:self.xdim])
						num = (neighbors.iloc[n].membership[c]) / (dist ** (2 / (m-1)))

						vote = num/den
						neighbors_votes.append(vote)
					votes[c] = np.sum(neighbors_votes)

				pred = max(votes.items(), key=operator.itemgetter(1))[0]
				y_pred.append((pred, votes))

			return y_pred


	def score(self, X, y):
		if self.fitted_ == None:
			raise Exception('score() called before fit()')
		else:
			predictions = self.predict(X)
			y_pred = [t[0] for t in predictions]
			confidences = [t[1] for t in predictions]

			return accuracy_score(y_pred=y_pred, y_true=y)


	def _find_k_nearest_neighbors(self, df, x):
		X = df.iloc[:,0:self.xdim].values

		df['distances'] = [np.linalg.norm(X[i] - x) for i in range(self.n)]

		df.sort_values(by='distances', ascending=True, inplace=True)
		neighbors = df.iloc[0:self.k]

		return neighbors


	def _get_counts(self, neighbors):
		groups = neighbors.groupby('y')
		counts = {group[1]['y'].iloc[0]:group[1].count()[0] for group in groups}

		return counts


	def _compute_memberships(self):
		memberships = []
		for i,j in zip(self.X,self.y):
			x = i
			y = j

			neighbors = self._find_k_nearest_neighbors(pd.DataFrame.copy(self.df), x)
			counts = self._get_counts(neighbors)

			membership = dict()
			for c in self.classes:
				try:
					uci = 0.49 * (counts[c] / self.k)
					if c == y:
						uci += 0.51
					membership[c] = uci
				except:
					membership[c] = 0

			memberships.append(membership)
		return memberships


	def _check_params(self, X, y):
		if type(self.k) != int:
			raise Exception('"k" should have type int')
		if self.k >= len(y):
			raise Exception('"k" should be less than no of feature sets')
		if self.k % 2 == 0:
			raise Exception('"k" should be odd')

		if type(self.plot) != bool:
			raise Exception('"plot" should have type bool')

In [ ]:
##Funciones relacionadas a los modelos
def show_results(y_test, pred_y, txt):
    print(txt, " METRICS:\n")
    conf_matrix = confusion_matrix(y_test, pred_y)
    plt.figure(figsize=(6, 6))
    sns.heatmap(conf_matrix, annot=True, fmt="d")
    plt.title("Confusion matrix")
    plt.ylabel('True class')
    plt.xlabel('Predicted class')
    plt.show()
    print (classification_report(y_test, pred_y))

# Representación de los datos de cada pasada con PYPLOT
def draw_results(history):
  ## Función de pérdida
  plt.figure(1, figsize=(5,3))
  plt.plot(history.history['loss'])
  plt.plot(history.history['val_loss'])
  plt.title('Función de pérdida')
  plt.ylabel('Error')
  plt.xlabel('epoch')
  plt.legend(['Entrenamiento', 'Validación'], loc='upper left')
  plt.grid()
  plt.show()
  
  ## Precisión (accuracy)
  plt.figure(2, figsize=(5,3))
  plt.plot(history.history['accuracy'])
  plt.plot(history.history['val_accuracy'])
  plt.title('Precisión del modelo')
  plt.ylabel('Precisión (accuracy)')
  plt.xlabel('epoch')
  plt.legend(['Entrenamiento', 'Validación'], loc='upper left')
  plt.grid()
  plt.show()

#get_class_weights(): utilizado para la obtención de los pesos de cada una de las clases asociadas al entrenamiento en un modelo de redes neuronales.
def get_class_weights(y, smooth_factor=0):
    """
    Returns the weights for each class based on the frequencies of the samples
    :param smooth_factor: factor that smooths extremely uneven weights
    :param y: list of true labels (the labels must be hashable)
    :return: dictionary with the weight for each class
    """
    counter = Counter(y)

    if smooth_factor > 0:
        p = max(counter.values()) * smooth_factor
        for k in counter.keys():
            counter[k] += p

    majority = max(counter.values())

    return {cls: float(majority / count) for cls, count in counter.items()}

"""
# nameModel: nombre del modelo que se utiliza. Opciones 'XGB' para XGBClassifier 
    o 'NN' para un modelo de redes neuronales
# model: la implementación del propio modelo. Opciones XGBClassifier o NN.
# balanced: si se quiere utilizar un balanceo de los pesos de las clases 
    (menor prioridad en las mayoritarias)
# oversamp: si se quieren generar datos sintéticos en las clases minoritarias 
    para evitar desequilibrio
# undersamp: si se quieren eliminar aleatoriamente datos de la clase mayoritaria
    para igualar el número total de muestras a las minoritarias
# ep: número de pasos del proceso de entrenamiento (epochs). Por defecto con 
  valor 10
# typeNN: tipo de red neuronal (1,2,3,4) que se quiere utilizar. 
  Creación y compilación antes del entrenamiento 
"""
def train_process(model, nameModel, balanced = False, oversamp = False, undersamp = False, ep = 10,k=8):
  features = 'IENSTFJP'
  
  # Estructura del modelo
  if nameModel == 'NN':
    model.summary()
    features = 'INTJ'
    
  d = 0;
  for i in features:
      print("%s ..." % (mbti[i]))
      
      # Entrenamiento de cada característica individualmente 
      Y_part = Y[i]

      # Separación del dataset en entrenamiento (train) y validación (test)
      seed = 7
      test_size = 0.40
      X_train, X_test, y_train, y_test = train_test_split(X_tfidf, Y_part, test_size=test_size, random_state=seed)

      #Se definen los conjuntos de validación y test desde el conjunto de test. 
      #Con relación a los datos originales, train es el 60%, test es el 25% y validation es el 15% 
      X_test,X_validation,y_test, y_validation = train_test_split(X_test, y_test, test_size=0.375, random_state=seed)
      print("y_train: ", Counter(y_train)) # y_train.value_counts()

      # Situación oversampling : VER LA DESCRIPCIÓN DE NOEMI UN POCOMÁS ARRIBA, LÍNEA 56 se generan muestras fake
      # estructura común en los dos modelos
      if oversamp:
        from imblearn.over_sampling import SVMSMOTE
        sm = SVMSMOTE(random_state=42)
        X_train_res, y_train_res = sm.fit_resample(X_train, y_train)        
        print ("Distribution before resampling {}".format(Counter(y_train)))
        print ("Distribution labels after resampling {}".format(Counter(y_train_res)))
        X_train = X_train_res;
        y_train = y_train_res;
      
      # Situación undersampling, estructura común en los dos modelos
      elif undersamp:
        from imblearn.under_sampling import ClusterCentroids
        cc = ClusterCentroids(random_state=0)
        X_resampled, y_resampled = cc.fit_resample(X_train, y_train)
        print("Undersampling: ", sorted(Counter(y_resampled).items()))
        X_train = X_resampled;
        y_train = y_resampled;

      ## MODELO XGBCLASSIFIER
      if nameModel == 'XGB':
        if balanced:  #sE BALANCEAN los pesos con la función weight_balance
          model.fit(X_train, y_train, sample_weight=compute_sample_weight("balanced", y_train),verbose=2)
        else:
          model.fit(X_train, y_train,verbose=2) # aqui son pesos desbalancedos, generados por defecto para el modelo
        
        # PRECISIÓN ENTRENAMIENTO (Train Accuracy)
        y_pred_train = model.predict(X_train)
        pred_train = [round(value) for value in y_pred_train]
        accuracy_train = accuracy_score(y_train, pred_train)
        print("* %s Train Accuracy: %.2f%%" % (mbti[i], accuracy_train * 100.0))

        # PRECISIÓN TEST (Test Accuracy)
        # Predicciones a partir de los datos de prueba
        y_pred = model.predict(X_test)
        predictions = [round(value) for value in y_pred]
        
        #PRECISIÓN DE VALIDACION (dataset no entrenado)..será siempre el más bajo en cuanto a accuracy
        #NOtar que los datos son totalmente nuevos para el modelo
        y_pred_val = model.predict(X_validation)
        predictions_validation = [round(value) for value in y_pred_val]

        # Evaluación de las predicciones, cálculo de la precisión en validación
        accuracy = accuracy_score(y_validation, predictions_validation)
        print("* %s Test Accuracy: %.2f%%" % (mbti[i], accuracy * 100.0))

        show_results(y_train, pred_train, 'TRAIN')
        show_results(y_test, predictions, 'TEST')  
        show_results(y_validation, predictions_validation, 'VALIDATION') 
      
      ## MODELO NN - NEURAL NETWORKS
      elif nameModel == 'NN':

        if balanced:
          #class_weights = class_weight.compute_class_weight('balanced', np.unique(y_train), y_train)
          weights = get_class_weights(y_train)
          print("Class_weights:", weights)
          history = model.fit(X_train, y_train, epochs=ep, batch_size=len(X_train), validation_data=(X_test, y_test), 
                              class_weight=weights)
        else:
          history = model.fit(X_train, y_train, epochs=ep, batch_size=len(X_train), validation_data=(X_test, y_test))

        d = d + 1
        
        # Evaluación del modelo (Accuracy) 
        print("Training")
        scores_train = model.evaluate(X_train, y_train, verbose=1) 
        print("\nAccuracy TRAIN: %.2f%%" % (scores_train[1]*100))
        print("Testing")
        scores = model.evaluate(X_test, y_test, verbose=1) 
        print("Accuracy TEST: %.2f%%" % (scores[1]*100))
        
        print(history)
        # Representación de resultados
        draw_results(history)
      
        y_pred_train = model.predict(X_train)
        pred_train = [np.round(value) for value in y_pred_train]
        show_results(y_train, pred_train, 'TRAIN')
        y_pred = model.predict(X_test)
        predictions = [np.round(value) for value in y_pred]
        show_results(y_test, predictions, 'TEST')
        y_pred = model.predict(X_validation)
        predictions = [np.round(value) for value in y_pred]
        show_results(y_validation, predictions, 'VALIDATION')
        
      
      elif nameModel == "KNN":
        for i in range(1,k):
          knn = KNeighborsClassifier(i)
          knn.fit(X_train, y_train)
          print("KNN con {} clases".format(i),end="\n")
          print('Accuracy of K-NN classifier on training set: {:.2f}'
              .format(knn.score(X_train, y_train)))
          print('Accuracy of K-NN classifier on test set: {:.2f}'
              .format(knn.score(X_test, y_test)))
          print("Se inicia predicción",end="\n")
          pred = knn.predict(X_validation)
          print(confusion_matrix(y_validation, pred))
          print(classification_report(y_validation, pred))
      
      elif nameModel == "fKNN":
        custModel = FuzzyKNN()        
        custModel.fit(X_train, y_train)
        print(cross_val_score(cv=5, estimator=custModel, X=X_test, y=y_test))
          
      print("--------------------------------------")

      del X_train, X_test, y_train, y_test, X_validation, y_validation

###**Modelo XGBOOST**

Tengo que explicar lo de balanceado y desbalanceado.

In [ ]:
#Se define el modelo
model = XGBClassifier()

Para datos desbalanceados

In [ ]:
train_process(model, 'XGB')

Para datos balanceados

In [ ]:
train_process(model, 'XGB', balanced=True)

Para datos balanceados, con oversampling

In [ ]:
train_process(model, 'XGB', balanced=False, oversamp=True)

A continuación se implementa  `XG boost Classifier` y se evalúa con la métrica `accuracy`.

In [ ]:
xgb = XGBClassifier()
xgb.fit(X_train,y_train)

Y_pred = xgb.predict(X_test)
predictions = [round(value) for value in Y_pred]

# Se evalúan las predicciones
accuracy = accuracy_score(y_test, predictions)

In [ ]:
accuracies['XG Boost'] = accuracy* 100.0

****comprar resutados, falta..... **texto en negrita**

In [ ]:
print("Accuracy: %.2f%%" % (accuracy * 100.0))

###**REDES NEURONALES**
Para los siguientes modelos se generan complicaciones determinadas  partir de la observación, así como de los resultados obtenidos por cada modelo.
Se sigue un patrón de diseño de redes tipo cascada, donde para cada modelo nuevo de la red, se genran complicaciones basadas en el modelo inmediato anterior.
Para todos los modelos subsecuentes se considera como métrica de bondad *accuracy*.
Se genera una clase para el balanceo de los pesos.

#####Definición de funciones

In [ ]:
import tensorflow.keras as keras

A continuación se definen los distintos optimizadpores que se serán utilizados en los modelos.

In [ ]:
lr = 0.01 # learning_rate (tasa de aprendizaje)

## Optimizador SGD
sgd = keras.optimizers.SGD(lr=lr)

## Optimizador ADAGRAD
adg = keras.optimizers.Adagrad(lr=lr, epsilon=None, decay=0.0)

## Optimizador RMSprop
rms = keras.optimizers.RMSprop(lr=lr, epsilon=None, decay=0.0)

#### **Modelo de Red Neuronal Número 1**
Se crea una Red Neuronal Recurrente con las siguientes características:


*   Una capa de entrada del tipo `Flatten` con  `Shape` de 2000
*   Una capa intermedia con una densidad de 32 neuronas y función de activación ReLU
*   Una capa de salida con densidad de 1 con función de activación *Sigmoide*
*   El optimizador es SGD 
*   La función de pérdida es Entropía Cruzada Binaria



In [ ]:
def create_compileNN():
  model1 = keras.models.Sequential()
  model1.add(keras.layers.Flatten(input_shape=(2000,))) # capa de entrada
  model1.add(keras.layers.Dense(32, activation='relu')) # capa oculta
  model1.add(keras.layers.Dense(1, activation='sigmoid')) # capa de salida
  
  # Compilación del modelo
  model1.compile(optimizer=sgd, 
                loss='binary_crossentropy', 
                metrics=['accuracy'])
  
  return model1

model1 = create_compileNN()

#####Primera mejora de hiperparámetros.

Para el primer proceso de entrenamiento se generan iteraciones con pasos de 2 unidades en el rango 5-19 que determinan el número de epochs.

In [ ]:
for i in range (5, 20, 2):
  print('NUMBER OF EPOCHS: ', i)
  train_process(model1, 'NN', ep=i)

#####Segunda mejora de hiperparámetros.

Para este segundo proceso de entrenamiento se generan iteraciones con pasos de 6 unidades (para tomar el valor medio)en el rango 7-19 que determinan el número de epochs.
Dado que en en la mejora anterior se obtuvo que los mejores valores están entre 7 y 19, se genera la validación del  modelo a partir de dicho rango.

In [ ]:
for i in range (7, 20, 6):
  print('NUMBER OF EPOCHS: ', i)
  train_process(model1, 'NN', ep=i)

#### **Modelo de Red Neuronal Número 2**

Se genera una segunda Red Neuronal de tipo Recurrente con las siguientes características:



*   Capa de entrada del tipo `Flatten` con `Shape` de 2000 (Igual que en la red anterior)
* Se genran n capas intermedias considerando densidades descritas en el arreglo *neuron*  con función de activación ReLU 
*   Capa de salida de tipo `Dense` con densidad de 1 y función de activación *Sigmoide*.
*   El optimizador es SGD 
*   La función de pérdida es Entropía Cruzada Binaria


In [ ]:
def create_compileNN(lyrs=1, neuron=[32]):
  model2 = keras.models.Sequential()
  model2.add(keras.layers.Flatten(input_shape=(2000,))) # capa de entrada
  for i in range (0, lyrs):
    model2.add(keras.layers.Dense(neuron[i], activation='relu')) # capa oculta i
  model2.add(keras.layers.Dense(1, activation='sigmoid')) # capa de salida

  model2.compile(optimizer=sgd, 
                loss='binary_crossentropy', 
                metrics=['accuracy'])
  return model2

Se generan 5 variaciones del modelo original donde se consideran 19 epochs (obtenidas del modelo anterior).

Se generan *complicaciones* al modelo:

1. Consta de 2 capas intermedias con densidad de 32 neuronas cada una.
2. Consta de 2 capas intemedias con 64 y 32 neuronas, esta red emula una red deconvolucional que por definición empieza por una densidad mayor a la con que se termina, considerando en todo momento las potencias de 2.
3. Consta de 2 capas intermedias con 64 neuronas cada una.
4. Consta de 3 capas intermedias con 32, 64 y 32 neuronas cada una. Esta capa en concreto emula una red convolucional-deconvolucional. 
5. Consta de 3 capas intermedia con 64,64 y 32 neuronas. Esta capa estaría emulando un red deconvolucional con una complicación específica; hablamos aquí de capas gemelas.

In [ ]:
epochs = 19

# CAPAS OCULTAS: 2 de 32
layers = 2; neuron = [32,32]
model2 = create_compileNN(layers, neuron)
train_process(model2, 'NN', ep=epochs)

# CAPAS OCULTAS: 2 de 64 y 32
layers = 2; neuron = [64,32]
model2 = create_compileNN(layers, neuron)
train_process(model2, 'NN', ep=epochs)

# CAPAS OCULTAS: 2 de 64
layers = 2; neuron = [64,64]
model2 = create_compileNN(layers, neuron)
train_process(model2, 'NN', ep=epochs)

# CAPAS OCULTAS: 3 de 32, 64 y 32
layers = 3; neuron = [32,64,32]
model2 = create_compileNN(layers, neuron)
train_process(model2, 'NN', ep=epochs)

# CAPAS OCULTAS: 3 de 64, 64 y 32
layers = 3; neuron = [64,64,32]
model2 = create_compileNN(layers, neuron)
train_process(model2, 'NN', ep=epochs)

####**Modelo de Red Neuronal Número 3**
Se genera una Red Neuronal Recurrente con las siguientes características:
* Capa de entrada del tipo Flatten con Shape de 2000.
* Las 3 capas ocultas constan con densidad 64,64,32 y función de activación ReLU.
*   Capa de salida de tipo `Dense` con densidad de 1 y función de activación *Sigmoide*.
*   Se generan 9 submodelos que se pueden observar en la segunda celda de este apartado:

  
1. Optimizador AdamGrad con 7 epochs.
2. Optimizador AdamGrad con 13 epochs.
3. Optimizador AdamGrad con 19 epochs.
4. Optimizador  RMSprop con 7 epochs.
5. Optimizador  RMSprop con 13 epochs.
6. Optimizador  RMSprop con 19 epochs.

*   La función de pérdida es Entropía Cruzada Binaria.


In [ ]:
# Modelo con dos capas ocultas, una con 64 y otra con 32 neuronas
def create_compileNN(optimizer):
  model3 = keras.models.Sequential()
  model3.add(keras.layers.Flatten(input_shape=(2000,))) # capa de entrada
  model3.add(keras.layers.Dense(64, activation='relu')) # capa oculta 1
  model3.add(keras.layers.Dense(64, activation='relu')) # capa oculta 2
  model3.add(keras.layers.Dense(32, activation='relu')) # capa oculta 3
  model3.add(keras.layers.Dense(1, activation='sigmoid')) # capa de salida

  model3.compile(optimizer=optimizer, 
                loss='binary_crossentropy', 
                metrics=['accuracy'])
  print('Modelo creado y compilado, ', optimizer)
  return model3

In [ ]:
epochs = [7, 13, 19]
opts = [adg, rms]
for i in opts:
  for j in epochs:
    model3 = create_compileNN(i)
    train_process(model3, 'NN', ep=j)

#### **Modelo de Red Neuronal Número 4**
Se genera una Red Neuronal Recurrente con las siguientes características:

* Capa de entrada del tipo Flatten con Shape de 2000.
* Consta de una capa tipo Dense con 64 neuronas y función de activación ReLU.
* Una capa Dropout con rango de pérdida entre 10% y 30%, dicha capa es alimentada a partir de un ciclo iterativo considerando los valores del rango.
* Dos capas ocultas de tipo Dense con 64 y 32 neuronas. Función de activación ReLU.
* Capa de salida del tipo Dense, con 1 neurona y función de activación Sigmoide.
* El optimizador es RMSProp
* La función de pérdida es Entropía Cruzada Binaria.

#####Variación Modelo 4a:

In [ ]:
# UNA CAPA DROPOUT AL INICIO 10%, 20%, 30%; 19 epochs
percen = [0.1, 0.2, 0.3]
i=0

def create_compileNN():
  model4 = keras.models.Sequential()
  model4.add(keras.layers.Flatten(input_shape=(2000,))) # capa de entrada
  model4.add(keras.layers.Dense(64, activation='relu')) # capa oculta 1
  model4.add(keras.layers.Dropout(percen[i])) # capa Dropout
  model4.add(keras.layers.Dense(64, activation='relu')) # capa oculta 2
  model4.add(keras.layers.Dense(32, activation='relu')) # capa oculta 3
  model4.add(keras.layers.Dense(1, activation='sigmoid')) # capa de salida

  model4.compile(optimizer=rms, 
                loss='binary_crossentropy', 
                metrics=['accuracy'])
  print('Modelo creado: Dropout ', percen[i])
  return model4

In [ ]:
# UNA CAPA DROPOUT AL INICIO 10%, 20%, 30%; 19 epochs
for i in range(0, len(percen)):
  model4 = create_compileNN()
  train_process(model4, 'NN', ep = 17)

#####Variación Modelo 4b:


In [ ]:
# DOS CAPAS DROPOUT DE 0.1
def create_compileNN():
  model4 = keras.models.Sequential()
  model4.add(keras.layers.Flatten(input_shape=(2000,))) # capa de entrada
  model4.add(keras.layers.Dense(64, activation='relu')) # capa oculta 1
  model4.add(keras.layers.Dropout(0.1)) # capa Dropout
  model4.add(keras.layers.Dense(64, activation='relu')) # capa oculta 2
  model4.add(keras.layers.Dropout(0.1)) # capa Dropout
  model4.add(keras.layers.Dense(32, activation='relu')) # capa oculta 3
  model4.add(keras.layers.Dense(1, activation='sigmoid')) # capa de salida

  model4.compile(optimizer=rms, 
                loss='binary_crossentropy', 
                metrics=['accuracy'])
  return model4

In [ ]:
# DOS CAPAS DROPOUT DE 0.1; 19 epochs
model4 = create_compileNN()
train_process(model4, 'NN', ep = 19)

#**Implementación de algoritmos en Soft Computing**
se abre la posibilidad de generar una mejora en los resultados obtenidos, con los modelos KNN y fKNN.

### **KNeighborsClassifier**
- Considerando las características de los modelos KNN se tomó la decisión de generar un modelo multiclase para la generación de los resultados en las predicciones de las características.
- Se evalúa con accuracy
- Se buscan los mejores parámetros con `scoreList`

In [ ]:
Y_part = Y[:]

In [ ]:
Y_part = Y_part.to_numpy()

In [ ]:
seed = 7
test_size = 0.40
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, Y_part, test_size=test_size, random_state=seed)

#Se definen los conjuntos de validación y test desde el conjunto de test. 
#Con relación a los datos originales, train es el 60%, test es el 25% y validation es el 15% 
X_test,X_validation,y_test, y_validation = train_test_split(X_test, y_test, test_size=0.375, random_state=seed)


#### KNN 8 clases
Se define un modelo con 8 clases y se entrena.

In [ ]:
knn = KNeighborsClassifier(8)
knn.fit(X_train, y_train)
print("KNN con {} clases".format(8),end="\n")
print('Accuracy of K-NN classifier on training set: {:.2f}'
    .format(knn.score(X_train, y_train)))
print('Accuracy of K-NN classifier on test set: {:.2f}'
    .format(knn.score(X_test, y_test)))
print("Se inicia predicción",end="\n")
pred = knn.predict(X_validation)

In [ ]:
print(classification_report(y_validation, pred)) #Se muestra el reporte de todas las clases
#enumaradas de 0 a 7, sin necesidad de usar lógica difusa

In [ ]:
lbls = [i for i in Y.columns]

Se define la matriz de confusión del algoritmo antes entrenado.

In [ ]:
cnfsm = multilabel_confusion_matrix(y_validation, pred)

Se crea una función que imprime los valores obtenidos tras la aplicación del modelo.

In [ ]:
import seaborn as sns
vis_arr = np.asarray(cnfsm)

def print_confusion_matrix(confusion_matrix, axes, class_label, class_names, fontsize=14):

    df_cm = pd.DataFrame(
        confusion_matrix, index=class_names, columns=class_names,
    )

    try:
        heatmap = sns.heatmap(df_cm, annot=True, fmt="d", cbar=False, ax=axes)
    except ValueError:
        raise ValueError("Confusion matrix values must be integers.")
    heatmap.yaxis.set_ticklabels(heatmap.yaxis.get_ticklabels(), rotation=0, ha='right', fontsize=fontsize)
    heatmap.xaxis.set_ticklabels(heatmap.xaxis.get_ticklabels(), rotation=45, ha='right', fontsize=fontsize)
    axes.set_ylabel('True label')
    axes.set_xlabel('Predicted label')
    axes.set_title("Confusion Matrix for the class - " + class_label)

fig, ax = plt.subplots(4, 2, figsize=(12, 9))
for axes, cnfsm, label in zip(ax.flatten(), vis_arr, lbls):
    print_confusion_matrix(cnfsm, axes, label, ["N", "Y"])

fig.tight_layout()
plt.show() #se generan las gráficas de sitorsión para cada una dde las clases de personlidad
#considerando valores predichos versus reales, con un eje de predicción y un eje de valor
#tiene 2 subejes binarios al valor de la predicción

#### KNN 16 clases
Se define un modelo con 8 clases y se entrena.

In [ ]:
#Se definen las clases
lbls
y_train_text=[]
y_test_text=[]
y_validation_text=[]

for i in y_validation: 
  temp=np.where(i==1)
  y_validation_text.append("".join(lbls[x] for x in temp[0]))

for i in y_train: 
  temp=np.where(i==1)
  y_train_text.append("".join(lbls[x] for x in temp[0]))

for i in y_test: 
  temp=np.where(i==1)
  y_test_text.append("".join(lbls[x] for x in temp[0]))

In [ ]:
#Se construye el modelo
knn = KNeighborsClassifier(16)
knn.fit(X_train, y_train_text)
print("KNN con {} clases".format(16),end="\n")
print('Accuracy of K-NN classifier on training set: {:.2f}'
    .format(knn.score(X_train, y_train_text)))
print('Accuracy of K-NN classifier on test set: {:.2f}'
    .format(knn.score(X_test, y_test_text)))
print("Se inicia predicción",end="\n")
pred = knn.predict(X_validation)

In [ ]:
#Se corroborán las predicciones
print(pred)

In [ ]:
print(classification_report(y_validation_text, pred)) #Se muestra el reporte de todas las clases


In [ ]:
cnfsm = multilabel_confusion_matrix(y_validation_text, pred)
vis_arr = np.asarray(cnfsm)

In [ ]:
fig, ax = plt.subplots(8, 2, figsize=(12, 24))

def print_confusion_matrix(confusion_matrix, axes, class_label, class_names, fontsize=14):

    df_cm = pd.DataFrame(
        confusion_matrix, index=class_names, columns=class_names,
    )

    try:
        heatmap = sns.heatmap(df_cm, annot=True, fmt="d", cbar=False, ax=axes)
    except ValueError:
        raise ValueError("Confusion matrix values must be integers.")
    heatmap.yaxis.set_ticklabels(heatmap.yaxis.get_ticklabels(), rotation=0, ha='right', fontsize=fontsize)
    heatmap.xaxis.set_ticklabels(heatmap.xaxis.get_ticklabels(), rotation=45, ha='right', fontsize=fontsize)
    axes.set_ylabel('True label')
    axes.set_xlabel('Predicted label')
    axes.set_title("Confusion Matrix for the class - " + class_label)


for axes, cnfsm, label in zip(ax.flatten(), vis_arr, set(y_train_text)):
    print_confusion_matrix(cnfsm, axes, label, ["N", "Y"])

fig.tight_layout()
plt.show() #se generan las gráficas de sitorsión para cada una dde las clases de personlidad
#considerando valores predichos versus reales, con un eje de predicción y un eje de valor
#tiene 2 subejes binarios al valor de la predicción

### **KNeighborsClassifier Difuso**


In [ ]:
 from sklearn.base import BaseEstimator, ClassifierMixin
 from sklearn.model_selection import cross_val_score
 import operator

In [ ]:
train_process(None, nameModel='fKNN' )